# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Feb 1 2023 8:37AM,256357,12,HH13123,Hush Hush,Released
1,Feb 1 2023 8:31AM,256354,12,HH8836,Hush Hush,Released
2,Feb 1 2023 8:30AM,256338,10,CTF0011611-BO,"Citieffe, Inc.",Released
3,Feb 1 2023 8:30AM,256342,10,0086344095,ISDIN Corporation,Released
4,Feb 1 2023 8:30AM,256342,10,0086344096,ISDIN Corporation,Released
5,Feb 1 2023 8:30AM,256342,10,0086344097,ISDIN Corporation,Released
6,Feb 1 2023 8:30AM,256342,10,0086344093,ISDIN Corporation,Released
7,Feb 1 2023 8:30AM,256342,10,0086344098,ISDIN Corporation,Released
8,Feb 1 2023 8:30AM,256342,10,0086344099,ISDIN Corporation,Released
9,Feb 1 2023 8:30AM,256342,10,0086344100,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
33,256351,Released,1
34,256352,Released,20
35,256353,Released,1
36,256354,Released,1
37,256357,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
256351,NaN,NaN,1.0
256352,NaN,NaN,20.0
256353,NaN,NaN,1.0
256354,NaN,NaN,1.0
256357,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
256259,9.0,0.0,3.0
256271,0.0,0.0,1.0
256274,0.0,0.0,1.0
256276,0.0,0.0,1.0
256280,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
256259,9,0,3
256271,0,0,1
256274,0,0,1
256276,0,0,1
256280,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,256259,9,0,3
1,256271,0,0,1
2,256274,0,0,1
3,256276,0,0,1
4,256280,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,256259,9,,3
1,256271,,,1
2,256274,,,1
3,256276,,,1
4,256280,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Feb 1 2023 8:37AM,256357,12,Hush Hush
1,Feb 1 2023 8:31AM,256354,12,Hush Hush
2,Feb 1 2023 8:30AM,256338,10,"Citieffe, Inc."
3,Feb 1 2023 8:30AM,256342,10,ISDIN Corporation
13,Feb 1 2023 8:30AM,256347,10,ISDIN Corporation
22,Feb 1 2023 8:30AM,256351,10,"Lupin Research, Inc."
26,Feb 1 2023 8:24AM,256353,19,"GCH Granules USA, Inc."
27,Feb 1 2023 7:39AM,256352,19,"AdvaGen Pharma, Ltd"
47,Jan 31 2023 4:50PM,256350,19,"GUSA Granules USA, Inc."
48,Jan 31 2023 4:00PM,256344,12,"SD Head USA, LLC"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Feb 1 2023 8:37AM,256357,12,Hush Hush,,,1
1,Feb 1 2023 8:31AM,256354,12,Hush Hush,,,1
2,Feb 1 2023 8:30AM,256338,10,"Citieffe, Inc.",,,1
3,Feb 1 2023 8:30AM,256342,10,ISDIN Corporation,,,15
4,Feb 1 2023 8:30AM,256347,10,ISDIN Corporation,3,,4
5,Feb 1 2023 8:30AM,256351,10,"Lupin Research, Inc.",,,1
6,Feb 1 2023 8:24AM,256353,19,"GCH Granules USA, Inc.",,,1
7,Feb 1 2023 7:39AM,256352,19,"AdvaGen Pharma, Ltd",,,20
8,Jan 31 2023 4:50PM,256350,19,"GUSA Granules USA, Inc.",,,1
9,Jan 31 2023 4:00PM,256344,12,"SD Head USA, LLC",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 1 2023 8:37AM,256357,12,Hush Hush,1,,
1,Feb 1 2023 8:31AM,256354,12,Hush Hush,1,,
2,Feb 1 2023 8:30AM,256338,10,"Citieffe, Inc.",1,,
3,Feb 1 2023 8:30AM,256342,10,ISDIN Corporation,15,,
4,Feb 1 2023 8:30AM,256347,10,ISDIN Corporation,4,,3
5,Feb 1 2023 8:30AM,256351,10,"Lupin Research, Inc.",1,,
6,Feb 1 2023 8:24AM,256353,19,"GCH Granules USA, Inc.",1,,
7,Feb 1 2023 7:39AM,256352,19,"AdvaGen Pharma, Ltd",20,,
8,Jan 31 2023 4:50PM,256350,19,"GUSA Granules USA, Inc.",1,,
9,Jan 31 2023 4:00PM,256344,12,"SD Head USA, LLC",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 1 2023 8:37AM,256357,12,Hush Hush,1,,
1,Feb 1 2023 8:31AM,256354,12,Hush Hush,1,,
2,Feb 1 2023 8:30AM,256338,10,"Citieffe, Inc.",1,,
3,Feb 1 2023 8:30AM,256342,10,ISDIN Corporation,15,,
4,Feb 1 2023 8:30AM,256347,10,ISDIN Corporation,4,,3


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 1 2023 8:37AM,256357,12,Hush Hush,1.0,NaN,NaN
1,Feb 1 2023 8:31AM,256354,12,Hush Hush,1.0,NaN,NaN
2,Feb 1 2023 8:30AM,256338,10,"Citieffe, Inc.",1.0,NaN,NaN
3,Feb 1 2023 8:30AM,256342,10,ISDIN Corporation,15.0,NaN,NaN
4,Feb 1 2023 8:30AM,256347,10,ISDIN Corporation,4.0,NaN,3.0


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 1 2023 8:37AM,256357,12,Hush Hush,1.0,0.0,0.0
1,Feb 1 2023 8:31AM,256354,12,Hush Hush,1.0,0.0,0.0
2,Feb 1 2023 8:30AM,256338,10,"Citieffe, Inc.",1.0,0.0,0.0
3,Feb 1 2023 8:30AM,256342,10,ISDIN Corporation,15.0,0.0,0.0
4,Feb 1 2023 8:30AM,256347,10,ISDIN Corporation,4.0,0.0,3.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,3075721,41.0,5.0,25.0
12,769055,3.0,0.0,0.0
16,768947,3.0,0.0,0.0
18,256276,1.0,0.0,0.0
19,2050506,37.0,7.0,8.0
20,256300,0.0,1.0,0.0
21,512633,1.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,3075721,41.0,5.0,25.0
1,12,769055,3.0,0.0,0.0
2,16,768947,3.0,0.0,0.0
3,18,256276,1.0,0.0,0.0
4,19,2050506,37.0,7.0,8.0
5,20,256300,0.0,1.0,0.0
6,21,512633,1.0,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,41.0,5.0,25.0
1,12,3.0,0.0,0.0
2,16,3.0,0.0,0.0
3,18,1.0,0.0,0.0
4,19,37.0,7.0,8.0
5,20,0.0,1.0,0.0
6,21,1.0,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,41.0
1,12,Released,3.0
2,16,Released,3.0
3,18,Released,1.0
4,19,Released,37.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,16,18,19,20,21
Status,,,,,,,
Completed,25.0,0.0,0.0,0.0,8.0,0.0,0.0
Executing,5.0,0.0,0.0,0.0,7.0,1.0,1.0
Released,41.0,3.0,3.0,1.0,37.0,0.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,16,18,19,20,21
0,Completed,25.0,0.0,0.0,0.0,8.0,0.0,0.0
1,Executing,5.0,0.0,0.0,0.0,7.0,1.0,1.0
2,Released,41.0,3.0,3.0,1.0,37.0,0.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,16,18,19,20,21
0,Completed,25.0,0.0,0.0,0.0,8.0,0.0,0.0
1,Executing,5.0,0.0,0.0,0.0,7.0,1.0,1.0
2,Released,41.0,3.0,3.0,1.0,37.0,0.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()